In [1]:
import random
import string
import os

import tensorflow as tf
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.models import load_model, Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization,Input,concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping,ReduceLROnPlateau , ModelCheckpoint
from keras.utils import plot_model
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

In [2]:

trainDir = "asl_alphabet_data"
batchSize = 64
targetSize = (64,64)

labels = []
alphabet = list(string.ascii_uppercase)
labels.extend(alphabet)
labels.extend(["del","nothing","space"])
print(labels)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
Num GPUs Available:  0


In [3]:
list_path = []
list_labels = []
for label in labels:
    label_path = os.path.join(trainDir, label, "*")
    image_files = glob.glob(label_path)

    sign_label = [label] * len(image_files)

    list_path.extend(image_files)
    list_labels.extend(sign_label)

metadata = pd.DataFrame({
    "image_path": list_path,
    "label": list_labels
})
metadata
X_train, X_test, y_train, y_test = train_test_split(
    metadata["image_path"], metadata["label"],
    test_size=0.15,
    random_state=2023,
    shuffle=True,
    stratify=metadata["label"]
)
data_train = pd.DataFrame({
    "image_path": X_train,
    "label": y_train
})

X_train, X_val, y_train, y_val = train_test_split(
    data_train["image_path"], data_train["label"],
    test_size=0.15/0.70,
    random_state=2023,
    shuffle=True,
    stratify=data_train["label"]
)
data_train = pd.DataFrame({
    "image_path": X_train,
    "label": y_train
})
data_val = pd.DataFrame({
    "image_path": X_val,
    "label": y_val
})
data_test = pd.DataFrame({
    "image_path": X_test,
    "label": y_test
})
print(data_train)
print(data_val)
print(data_test)

                                     image_path    label
51901              asl_alphabet_data\R\R181.jpg        R
51887             asl_alphabet_data\R\R1798.jpg        R
5050              asl_alphabet_data\B\B2844.jpg        B
29446               asl_alphabet_data\J\J50.jpg        J
58405             asl_alphabet_data\T\T2263.jpg        T
...                                         ...      ...
47111               asl_alphabet_data\P\P29.jpg        P
83611  asl_alphabet_data\nothing\nothing649.jpg  nothing
67329             asl_alphabet_data\W\W2195.jpg        W
85296     asl_alphabet_data\space\space2165.jpg    space
60158             asl_alphabet_data\U\U1140.jpg        U

[58103 rows x 2 columns]
                          image_path label
24961  asl_alphabet_data\I\I1864.jpg     I
71053  asl_alphabet_data\X\X2847.jpg     X
73637  asl_alphabet_data\Y\Y2472.jpg     Y
72875  asl_alphabet_data\Y\Y1787.jpg     Y
40669  asl_alphabet_data\N\N2500.jpg     N
...                             

In [4]:
#data augmentation
# takes data on the fly and generate its variant with different parameters here rescale then those data is fed from the directory
def data_aug():
    train_generator = ImageDataGenerator(rescale=1. / 255).flow_from_dataframe(
        data_train,
        directory='./',
        target_size=targetSize,
        x_col="image_path",
        y_col="label",
        batch_size=batchSize,
        class_mode='categorical',
        # color_mode='grayscale',
    )

    validation_generator = ImageDataGenerator(rescale=1. / 255).flow_from_dataframe(
        data_val,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=targetSize,
        batch_size=batchSize,
        class_mode='categorical',
        # color_mode='grayscale',
    )

    test_generator = ImageDataGenerator(rescale=1. / 255).flow_from_dataframe(
        data_test,
        directory='./',
        x_col="image_path",
        y_col="label",
        target_size=targetSize,
        batch_size=batchSize,
        class_mode='categorical',
        use_multiprocessing=True,
        workers=6,
        # color_mode='grayscale',
        shuffle=False,
    )
    return train_generator , validation_generator , test_generator



def seed_set(seed : int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_set(2023)

train_generator , validation_generator , test_generator = data_aug()

Found 58103 validated image filenames belonging to 29 classes.
Found 15847 validated image filenames belonging to 29 classes.
Found 13050 validated image filenames belonging to 29 classes.


In [6]:
base_model = VGG16(weights="imagenet" , include_top=False, input_shape=(64,64,3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(29, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

# earlyStop = EarlyStopping(monitor='val_loss', patience=10)
earlyStop = EarlyStopping(monitor='val_loss',min_delta=0.001,patience= 5,restore_best_weights= True,verbose = 0)

reduceLearn = ReduceLROnPlateau(monitor='val_accuracy',patience = 2,factor=0.5 ,verbose = 1)

model.compile(optimizer='adam', loss = 'categorical_crossentropy' , metrics=['accuracy'])

checkpoint = ModelCheckpoint('best_weight.keras', save_best_only=True, monitor='val_accuracy', mode='max')


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 64, 64, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 16, 16, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 8, 8, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 16,041,309 (61.19 MB)

 Trainable params: 1,326,621 (5.06 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [ ]:
# fitting model
with tf.device('/GPU:0'):
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batchSize,
        epochs=5,
        validation_data=validation_generator,
        verbose=2,
        validation_steps=validation_generator.samples // batchSize,
        callbacks=[ earlyStop , reduceLearn , checkpoint,tensorboard_callback],
    )


Epoch 1/5


In [ ]:

#saving model
modelJson = model.to_json()
with open("previousTrainedModel/99/modelVersion2.0.4.json", 'w') as json_file:
    json_file.write(modelJson)
model.save("modelVersion2.0.4.h5")


#evaluating
loss,acc = model.evaluate(train_generator , verbose = 0)

scores = model.evaluate(test_generator)
print("%s: %.2f%%" % ("Evaluate Test Accuracy", scores[1]*100))